# Install

# Import

In [ ]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import re,os,json
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)

# fn

In [ ]:
from modules.schedule import ITR1Sections,ITR1BatchProcessor

# Process ONE PDF

In [ ]:
input_file_path = 'OUTPUT/ ITR 1 AY 2023-24.pdf'
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
config_path = "config/ITR1_header.json"
itr = ITR1Sections(input_file_path,output_file_path,config_path)
print(itr.pan)
print(itr.ack)

In [ ]:
itr.get_section("SCHEDULE_TDS1")

# Process ALL PDFs

In [ ]:
input_dir="OUTPUT"
config_path = "config/ITR1_header.json"
processor = ITR1BatchProcessor(input_dir, config_path)
processor.process_all()

In [ ]:
metadata_df = processor.metadata()
metadata_df

In [ ]:
itr1_0=metadata_df.loc[0, "itr_obj"]
itr1_0

In [ ]:
itr1_0.debug["section_ranges"]

In [ ]:
# processor.export_by_pan()